In [ ]:
# Descrição
# Este script faz uma análise exploratório do dataset processado com os registros dentro da 
# área de são paulo e que estão entre 6 e 22:59:59
#
# Description
# This script analyzes the processed dataset (register in Sao Paulo region between 6-23 hours)
#

In [ ]:
# Spark Config
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing necessary packages for notebook session
sc.install_pypi_package("boto3")

In [ ]:
from pyspark.sql import functions as F
import boto3

# new file header
csv_out = "day,line_id_distinct,id_avl_distinct,line-id_id-avl_distinct,event_0_count,event_64_count,id_point_distinct,hour_diff_mean,hour_diff_min,hour_diff_max,hour_diff_stddev,hour_diff_quantile_25,hour_diff_quantile_50,hour_diff_quantile_75\n"

# from october 1 to october 31 
for day in range(1,32):

    # reading files
    traces = spark.read.parquet("s3a://mobility-traces-sp/processed-data/using-server-hour/records-between-6-23-only-sp-server-hour/MO_1510"+str(day) +  "/")

    # counting distinct line_id
    line_id_distinct = traces.agg(F.countDistinct("line_id")).collect()[0][0]

    # counting distinct id_avl
    id_avl_distinct = traces.agg(F.countDistinct("id_avl")).collect()[0][0]

    # counting distinct pairs line_id,id_avl
    lines_avl_distinct = traces.select('line_id','id_avl').distinct().count()

    # counting the numbers of registers with 0 in field "event"
    number_0 = traces.filter("event == 0").count()

    # counting the numbers of registers with 64 in field "event"
    number_64 = traces.filter("event == 64").count()

    # counting the numbers of distinct id_point
    id_point_distinct = traces.agg(F.countDistinct("id_point")).collect()[0][0]

    # getting hour_diff info (mean,min,max,stddev)
    hour_diff_info = traces.agg(F.mean('hour_diff').alias('mean'),
                       F.min('hour_diff').alias('min'),
                       F.max('hour_diff').alias('max'),
                       F.stddev('hour_diff').alias("stddev")).collect()

    # getting hour_diff quantiles (25%, 50%, 75%)
    # 0.0001 is the precision
    hour_diff_quantile = traces.approxQuantile("hour_diff", [0.25,0.5,0.75], 0.0001)

    csv_out += "MO_1510" + str(day) +","+ str(line_id_distinct) + ","  + str(id_avl_distinct)+ "," + str(lines_avl_distinct) + "," + str(number_0) + "," + str(number_64) + "," + str(id_point_distinct)+ "," + str(hour_diff_info[0]["mean"]) + "," + str(hour_diff_info[0]["min"]) + "," + str(hour_diff_info[0]["max"]) + "," + str(hour_diff_info[0]["stddev"]) + "," + str(hour_diff_quantile[0]) + "," + str(hour_diff_quantile[1]) + "," + str(hour_diff_quantile[2]) + "\n"
s3 = boto3.client('s3')

# writing results in S3
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/exploring-data/1-general-statistics.csv')

In [ ]:
# Another way to extract quantiles of data --> using spark sql approx percentile
# from pyspark.sql import SQLContext
# sqlContext = SQLContext(sc)
# traces.registerTempTable("df")
# df = sqlContext.sql("SELECT approx_percentile(hour_diff, 0.25,1000000) FROM df").collect()

In [ ]:
# rounding data 2 fields after comma to fits in excel precision 
from pyspark.sql.types import *

# schema of the raw data
custom_schema = StructType([
    StructField("day", StringType()),
    StructField("line_id_distinct", IntegerType()),
    StructField("id_avl_distinct", IntegerType()),
    StructField("line-id_id-avl_distinct", IntegerType()),
    StructField("event_0_count", IntegerType()),
    StructField("event_64_count", IntegerType()),
    StructField("hour_diff_mean", DoubleType()),
    StructField("hour_diff_min", DoubleType()),
    StructField("hour_diff_max", DoubleType()),
    StructField("hour_diff_stddev", DoubleType()),
    StructField("hour_diff_quantile_25", DoubleType()),
    StructField("hour_diff_quantile_50", DoubleType()),
    StructField("hour_diff_quantile_75", DoubleType())
])

# reading file
general_stats = spark.read.csv("s3a://mobility-traces-sp/statistics/exploring-data/1-general-statistics.csv",header="true",schema=custom_schema)

In [ ]:
# Adapting file with float with 2 fields, because of excel visualization support 
import numpy as np
from pyspark.sql.types import *

# reading file
lines_rdd = sc.textFile("s3a://mobility-traces-sp/statistics/exploring-data/1-general-statistics.csv")

#
# people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

# # Infer the schema, and register the DataFrame as a table.
# schemaPeople = spark.createDataFrame(people)
# schemaPeople.createOrReplaceTempView("people")

# reading first line aka header
header = lines_rdd.first()

# eliminating header
new_lines_rdd = lines_rdd.filter(lambda line: line != header)

# splitting lines by ,
splitted = new_lines_rdd.map(lambda l: l.split(","))

# transforming data with map lambda function
transformed = splitted.map(lambda l: (l[0],l[1],l[2],l[3],l[4],l[5],l[6],float(np.round(float(l[7]),2)),float(np.round(float(l[8]),2)),float(np.round(float(l[9]),2)),float(np.round(float(l[10]),2)),float(np.round(float(l[11]),2)),float(np.round(float(l[12]),2)),float(np.round(float(l[13]),2))))

# creating dataframe from the new transformed data
new_df = spark.createDataFrame(transformed)

# file header
columns_names = ["day","line_id_distinct","id_avl_distinct","line-id_id-avl_distinct","event_0_count","event_64_count","id_point_distinct","hour_diff_mean","hour_diff_min","hour_diff_max","hour_diff_stddev","hour_diff_quantile_25","hour_diff_quantile_50","hour_diff_quantile_75"]

for i in range(0,len(columns_names)):
    new_df = new_df.withColumnRenamed("_%s" % (i + 1), columns_names[i])
    
# write file in S3
new_df.repartition(1).write.option("header","true").csv("s3://mobility-traces-sp/statistics/exploring-data/1-general-statistics-excel-version")

In [ ]:
# https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html

In [ ]:
#https://forums.wikitechy.com/question/how-to-find-median-and-quantiles-using-spark/
# quantile splits data in equal sized groups
# percentile = splits the data in 100 equally sized groups
# https://towardsdatascience.com/exploratory-data-analysis-eda-with-pyspark-on-databricks-e8d6529626b1

In [2]:
# Result
# Saved file header/fields
# day --> day of the month (october)
# line_id_distinct --> number of distinct bus line in a day
# id_avl_distinct --> number of distinct bus in a day
# line-id_id-avl_distinct --> distinct line_id/bus_id pair
# event_0_count --> number of event 0
# event_64_count --> number of event 64
# hour_diff_mean --> mean of difference between hour_server and hour_avl
# hour_diff_min --> min value of difference between hour_server and hour_avl
# hour_diff_max --> max value of difference between hour_server and hour_avl
# hour_diff_stddev --> std value of difference between hour_server and hour_avl
# hour_diff_quantile_25 --> hour_diff < x of 25% of registers in that day
# hour_diff_quantile_50 --> hour_diff < x of 50% of registers in that day
# hour_diff_quantile_75 --> hour_diff < x of 75% of registers in that day